Recopilación Datos

Tomaremos datos y los guardaremos en un csv siempre que los resultados tengan un error relativo menor a 0.1%

In [1]:
using LinearAlgebra
using DifferentialEquations
using DelimitedFiles

function phi(Point1::Array{Float64, 1}, Point2::Array{Float64, 1}, v_wind::Array{Float64, 1}, tau) 
            #tgphi= (Point2[2]-Point1[2]- v_wind[2]*tau)/(Point2[1]-Point1[1]- v_wind[1]*tau) #Cálculo phi primera iteración
            #phi=rad2deg(atan(tgphi))
            phi=rad2deg(atan((Point2[2]-Point1[2]- v_wind[2]*tau),(Point2[1]-Point1[1]- v_wind[1]*tau)))
            return phi
end
function Slowness_Spherical(Point1::Array{Float64, 1}, Point2::Array{Float64, 1}, v_wind::Array{Float64, 1}, phi, c_source)
    x1,y1,z1=Point2-Point1 #Obtención parámetros primera iteración
    r=norm(Point2-Point1)
    theta=rad2deg(acos(z1/r))
    q0=1/(c_source + (v_wind[1]*cosd(phi)+ v_wind[2]*sind(phi))*cosd(theta) )
    qx=q0*cosd(phi)*sind(theta)
    qy=q0*sind(phi)*sind(theta)
    qz=q0*cosd(theta)
    return q0,qx,qy,qz,theta
end
function c_z(T,z) 
    if z>0
        c_air=331*sqrt((T+273.15-z/200)/273.15) #Modelo termodinámico v_sonido.Introduzco T_Z0 en grados centígrados por comodidad
    else
        c_air=331*sqrt((T+273.15)/273.15)
    end
    return c_air
end
function g!(dg, g, p, t)
    qx,qy = p
    q_aux = sqrt(qx^2 + qy^2 + g[4]^2)
    c=c_z(T,g[3])
    dg[1] = c*qx/q_aux + v_wind[1] #Variación coordenada X
    dg[2] = c*qy/q_aux + v_wind[2]  #Variación coordenada Y
    dg[3] = c*g[4]/q_aux + v_wind[3] #Variación coordenada Z
    dg[4]=+q_aux*(331^2/(400*273.15))/c #Variación qz
end


E_rel_ideal=  0.01
v_wind = [-20 + (20 - (-20)) * rand(),-20 + (20 - (-20)) * rand(),-20 + (20 - (-20)) * rand()]
Source = [10000 * rand(), 10000 * rand() , 10000 * rand()]
Receiver = [10000 * rand(), 10000 * rand() , 10000 * rand()]
T = rand(-20.:40.)

#### Necesitamos una velocidad inicial, correspondiente a la fuente, y un ángulo azimutal para empezar a iterar.
c_source=c_z(T, Source[3])
####
R=norm(Receiver-Source)
t_min=R/c_z(T,min(Source[3],Receiver[3]))
t_max=R/c_z(T, max(Source[3],Receiver[3]))
phi1=phi(Source,Receiver,v_wind, t_max) #Acotación teórica phi
phi2=phi(Source,Receiver,v_wind, t_min) #Acotación teórica phi
phi_i=(phi1+phi2)/2 #La media de la cota de phi como phi para la primera iteración
#######
q0,q_x,q_y,q_z,theta_i=Slowness_Spherical(Source,Receiver,v_wind, phi_i, c_source) #Obtención parámetros
theta_recta=deepcopy(theta_i)
#######
traveltime=[]
detener = 0 # Variable de control
for l in 1:15
    if detener==1
        break
    end     
    thetas= range(theta_i-45/(2^(l-1)), theta_i+45/(2^(l-1)), 50);
    if v_wind[1:2]==[0.,0.]
            phis=[deepcopy(phi_i)]
    else
        if l==1
            phis=(min(phi1,phi2):0.05:max(phi1,phi2))
        else
            phi_distance=max(phi1,phi2)-min(phi1,phi2)
            phis= range(phi_i-phi_distance/(2^(l)), theta_i-phi_distance/(2^(l)), 20);
        end
    end
    angulos_theta = []
    angulos_phi = []
    tiempos = []
    for i in 1:length(thetas)
        for j in 1:length(phis)
            if l>1
                q0=1/(c_source + (v_wind[1]*cosd(phi_i)+ v_wind[2]*sind(phi_i))*cosd(theta_i) )
            end
            q_x=q0*cosd(phis[j])*sind(thetas[i])
            q_y=q0*sind(phis[j])*sind(thetas[i])
            q_z=q0*cosd(thetas[i])
            p = [q_x, q_y]
            u0 = [Source[1],Source[2],Source[3], q_z]
            tspan=[0,t_max]
            prob = ODEProblem(g!, u0, tspan, p)
            paso=1e-2*R/2000
            sol = solve(prob, RK4(), dt=paso, saveat=0:paso:t_max)
            for k in 1:length(sol.u)
                if detener==0
                    if sqrt((sol.u[k][1] - Receiver[1])^2 +(sol.u[k][2] - Receiver[2])^2 + (sol.u[k][3] - Receiver[3])^2 ) < R/(20*2^(l-1))
                        push!(angulos_theta, thetas[i])
                        push!(angulos_phi, phis[j])
                        push!(tiempos, sol.t[k])
                        break
                    elseif sqrt((sol.u[k][1] - Receiver[1])^2 +(sol.u[k][2] - Receiver[2])^2 + (sol.u[k][3] - Receiver[3])^2 ) < R*E_rel_ideal/100
                        traveltime = sol.t[k] #Error relativo menor a 0.01% ya detenemos el barrido
                        theta_i = thetas[i]
                        phi_i = phis[j]
                        detener = 1 # Cambiamos el valor de la variable de control
                        println("Se alcanzó el error relativo objetivo.")
                        break
                    end
                else
                    break
                end
            end
        end
    end
    if length(tiempos)==0
        if l==1
            println("No se encontró ninguna trayectoria entre fuente y receptor. Valora si estás en una zona de sombra o ajusta el paso de tiempo ")
        end
        break
    elseif detener==0
        traveltime = findmin(tiempos)
        indice = argmin(tiempos)
        theta_i=angulos_theta[indice]
        phi_i=angulos_phi[indice]
    end
end
q_x=q0*cosd(phi_i)*sind(theta_i)
q_y=q0*sind(phi_i)*sind(theta_i)
q_z=q0*cosd(theta_i)
# Definir los parámetros
p = [q_x, q_y]
u0 = [Source[1],Source[2], Source[3], q_z]
tspan=[0,traveltime[1]]
# Crear el problema ODE
prob = ODEProblem(g!, u0, tspan, p)
# Resolver el problema ODE
paso=1e-2*R/2000
sol = solve(prob, RK4(), dt=paso, saveat=0:paso:traveltime[1])
println("El traveltime es: ", traveltime[1], " s.") 
println("Se reconstruye la trayectoria con ángulos de emisión: polar θ=", theta_i, "º y azimutal φ=", phi_i, "º.")
error_relativo=(sqrt((sol.u[end][1] - Receiver[1])^2 +(sol.u[end][2] - Receiver[2])^2 + (sol.u[end][3] - Receiver[3])^2 )/R)*100
println("El error relativo es del ", error_relativo, " %.")
#### Datos a recopilar
Delta_X=Receiver[1]-Source[1]
Delta_Y=Receiver[2]-Source[2]
Delta_Z=Receiver[3]-Source[3]
W_X=v_wind[1]
W_Y=v_wind[2]
W_Z=v_wind[3]
Delta_Theta=theta_i-theta_recta
phi_recta=phi(Source,Receiver,v_wind, traveltime[1])
Delta_phi=phi_i-phi_recta
####
if error_relativo<0.2  #Es en tanto por ciento 
    new_row=[Source[1],Source[2],Source[3],Delta_X, Delta_Y, Delta_Z, W_X, W_Y, W_Z, T, phi_recta , phi_i, Delta_phi, theta_recta, theta_i, Delta_Theta, error_relativo]
  # Convierte la fila en un array bidimensional
    new_row_2d = reshape(new_row, 1, length(new_row))
    # Añade la nueva fila al archivo
    open("C:\\Users\\Miguel\\Desktop\\TFM\\Recopilación_Datos.txt", "a") do io
        writedlm(io, new_row_2d, ',')
    end
end  
    

El traveltime es: 9.102595705083049 s.
Se reconstruye la trayectoria con ángulos de emisión: polar θ=116.9324489231651º y azimutal φ=-38.24003189453898º.
El error relativo es del 4.880226977300053 %.


No encontró ninguna trayectoria para Source=[6455,8184,6252], Receiver=[2986,9823,6306],v_wind=[11,-13,3], T=24
No encontró ninguna trayectoria para Source=[4427,6774,5789], Receiver=[6906,2379,5206],v_wind=[-16,19,-11], T=-18

Tenemos 172 datos para entrenar el modelo. Dado que theta y phi correspondientes a la línea recta se pueden conocer del resto de características, las eliminamos para el entrenamiento. Además, la columna correspondiente al error relativo nos da una idea de la validez de cada dato de entrenamiento, por lo que podemos asociar un peso a cada fila de 1-E_rel

In [2]:
using Pkg
Pkg.add("MLJ") #Machine Learning para Julia
Pkg.add("GLM")
Pkg.add("MLJModels")
Pkg.add("MLJGLMInterface")
Pkg.add("MLJMultivariateStatsInterface")
Pkg.add("DecisionTree")
Pkg.add("MLJDecisionTreeInterface")
Pkg.add("EvoTrees")
Pkg.add("Distributions")

    Updating registry at `C:\Users\Miguel\.julia\registries\General.toml`
   Resolving package versions...
   Installed EarlyStopping ──────────── v0.3.0
   Installed Libuuid_jll ────────────── v2.40.1+0
   Installed Expat_jll ──────────────── v2.6.2+0
   Installed StatisticalTraits ──────── v3.2.0
   Installed Cairo_jll ──────────────── v1.18.0+2
   Installed InverseFunctions ───────── v0.1.14
   Installed OpenEXR_jll ────────────── v3.2.4+0
   Installed Xorg_libICE_jll ────────── v1.1.1+0
   Installed MLJModels ──────────────── v0.16.17
   Installed ScientificTypesBase ────── v3.0.0
   Installed MLJTuning ──────────────── v0.7.4
   Installed StrideArraysCore ───────── v0.5.6
   Installed FixedPointNumbers ──────── v0.8.5
   Installed RecursiveFactorization ─── v0.2.22
   Installed IntervalArithmetic ─────── v0.22.11
   Installed LatinHypercubeSampling ─── v1.9.0
   Installed JpegTurbo_jll ──────────── v3.0.3+0
   Installed Libmount_jll ───────────── v2.40.1+0
   Installed Polyester ─

        Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated -m`
Precompiling project...
  ✓ Permutations
  ✓ ScientificTypesBase
  ✓ FillArrays
  ✓ OffsetArrays
  ✓ SentinelArrays
  ✓ Compat
  ✓ InverseFunctions
  ✓ EarlyStopping
  ✓ StableRNGs
  ✓ PtrArrays
  ✓ PrettyPrinting
  ✓ TranscodingStreams
  ✓ FastLapackInterface
  ✓ MutableArithmetics
  ✓ Latexify
  ✓ LossFunctions
  ✓ GeoInterface
  ✓ Krylov
  ✓ Libmount_jll
  ✓ FixedPointNumbers
  ✓ CPUSummary
  ✓ Xorg_libICE_jll
  ✓ Imath_jll
  ✓ IntelOpenMP_jll
  ✓ LAME_jll
  ✓ Pixman_jll
  ✓ JpegTurbo_jll
  ✓ oneTBB_jll
  ✓ Expat_jll
  ✓ LZO_jll
  ✓ Libgpg_error_jll
  ✓ Libuuid_jll
  ✓ FriBidi_jll
  ✓ StatisticalTraits
  ✓ ZMQ
  ✓ FillArrays → FillArraysPDMatsExt
  ✓ IntervalArithmetic
  ✓ Compat → CompatLinearAlgebraExt
  ✓ OffsetArrays → OffsetArraysAdaptExt
  ✓ InverseFunctions → Da

  ✓ LineSearches
  ✓ Distributions
  ✓ HistogramThresholding
  ✓ ImageTransformations
  ✓ ImageAxes
  ✓ TiledIteration
  ✓ ImageShow
  ✓ CloseOpenIntervals
  ✓ LayoutPointers
  ✓ SparseDiffTools
  ✓ NLsolve
  ✓ PlotlyJS → DataFramesExt
  ✓ ImageBinarization
  ✓ Distributions → DistributionsTestExt
  ✓ ImageMetadata
  ✓ ImageContrastAdjustment
  ✓ StrideArraysCore
  ✓ VectorizationBase
  ✓ Gridap
  ✓ PlotlyJS → IJuliaExt
  ✓ SimpleRandom
  ✓ MonteCarloIntegration
  ✓ ScientificTypes
  ✓ Plots
  ✓ KernelDensity
  ✓ Polyester
  ✓ Netpbm
  ✓ MathOptInterface
  ✓ SLEEFPirates
  ✓ Integrals
  ✓ CategoricalDistributions
  ✓ Plotly
  ✓ FastBroadcast
  ✓ ImageFiltering
  ✓ Plots → FileIOExt
  ✓ SparseDiffTools → SparseDiffToolsPolyesterExt
  ✓ ImageIO
  ✓ Gadfly
  ✓ Optim
  ✓ Integrals → IntegralsForwardDiffExt
  ✓ Ipopt
  ✓ MLJModels
  ✓ FEniCS
  ✓ MLJBase
  ✓ DiffEqBase
  ✓ JuMP
  ✓ SimpleGraphs
  ✓ Plots → UnitfulExt
  ✓ ImageCorners
  ✓ MLJIteration
  ✓ LoopVectorization
  ✓ MLJTuning
  ✓ M

In [1]:
using CSV, DataFrames, MLJ

# Cargar los datos
df = CSV.read("C:\\Users\\Miguel\\Desktop\\TFM\\Recopilación_Datos.txt", DataFrame)
println(names(df))


["Source_X(m)", "Source_Y(m)", "Source_Z(m)", "Delta_X(m)", "Delta_Y(m)", "Delta_Z(m)", "w_x(m/s)", "w_y(m/s)", "w_z(m/s)", "T(ºC)", "Phi_línea_recta(º)", "Phi_real(º)", "Delta_Phi(º)", "Theta_línea_recta(º)", "Theta_real(º)", "Delta_Theta(º)", "Error_Relativo(%)"]


In [2]:
#Hacerlo para modelo que soporte pesos
# Convertir el error relativo a pesos
df."ValidezFilas" = 1 .- df."Error_Relativo(%)" ./ 100

# Eliminar las columnas redundantes. 
#select!(df, Not([:"T(ºC)",:"w_x(m/s)",:"w_y(m/s)",:"w_z(m/s)",:"Phi_línea_recta(º)", :"Error_Relativo(%)", :"Theta_línea_recta(º)"]))
select!(df, Not([:"Phi_línea_recta(º)", :"Error_Relativo(%)", :"Theta_línea_recta(º)"]))

Row,Source_X(m),Source_Y(m),Source_Z(m),Delta_X(m),Delta_Y(m),Delta_Z(m),w_x(m/s),w_y(m/s),w_z(m/s),T(ºC),Phi_real(º),Delta_Phi(º),Theta_real(º),Delta_Theta(º),ValidezFilas
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,9887.0,9161.0,4542.0,-5821.0,-1918.0,359.0,-8.0,-2.0,-9.0,-3.0,-161.65,0.00231758,86.6522,0.00448422,0.999915
2,8821.0,3914.0,5791.0,-324.0,2847.0,-1893.0,17.0,15.0,12.0,-15.0,100.782,-0.0428599,127.425,3.97481,0.99929
3,790.0,1100.0,7589.0,1034.0,36.0,521.0,14.0,9.0,-13.0,21.0,0.188289,-0.0471987,60.2585,-3.01339,0.998518
4,6741.0,7116.0,7254.0,-466.0,2644.0,-3013.0,19.0,12.0,-1.0,-20.0,105.997,-0.0882664,140.048,1.75064,0.99883
5,9510.0,8714.0,2331.0,-2413.0,-5550.0,-1591.0,3.0,2.0,-19.0,-7.0,-113.86,-0.0066346,103.022,-1.70759,0.999221
6,8777.0,1157.0,1200.0,-5071.0,5979.0,4667.0,3.0,-15.0,15.0,21.0,128.827,-0.0386552,64.7596,5.52455,0.999281
7,9575.0,4578.0,1617.0,-4042.0,-1937.0,4341.0,12.0,-1.0,11.0,38.0,-155.804,-0.0488249,49.5613,3.64477,0.998704
8,9789.0,364.0,3751.0,-2693.0,615.0,1618.0,-4.0,10.0,-3.0,16.0,168.947,-0.00698519,59.3683,-0.27264,0.999541
9,3042.0,7948.0,649.0,4409.0,-270.0,1629.0,-7.0,-16.0,16.0,21.0,-0.648647,-0.0459717,73.7462,3.98916,0.998844


Primero para Delta_Theta

In [10]:
using MLJ

# Definir las columnas de destino
target_cols_1 = [ :"Delta_Theta(º)"]
ex_cols= [ :"ValidezFilas"]
# Definir las columnas de entrada (todas las columnas que no son de destino)
input_cols_1 = setdiff(names(df), target_cols_1)

# Excluir la columna 'ValidezFilas' de las columnas de entrada
input_cols_1 = setdiff(input_cols_1,ex_cols )

# Seleccionar manualmente las columnas de entrada y de destino
X = select(df, input_cols_1)
y = select(df, target_cols_1)

# Dividir los datos en conjuntos de entrenamiento y prueba
train, test = partition(1:nrow(df), 0.7, shuffle=true) # 70% para entrenamiento



([52, 122, 139, 102, 153, 65, 169, 55, 121, 137  …  167, 18, 125, 22, 152, 59, 44, 80, 134, 46], [15, 17, 47, 135, 116, 73, 98, 5, 147, 170  …  61, 16, 8, 123, 10, 154, 68, 60, 112, 64])

In [11]:
using MLJ, GLM , MLJGLMInterface
# Seleccionar el modelo
model = @load LinearRegressor pkg=GLM

# Crear una instancia del modelo
model_instance = LinearRegressor()

import MLJGLMInterface ✔


[ Info: For silent loading, specify `verbosity=0`. 


LinearRegressor(
  fit_intercept = true, 
  dropcollinear = false, 
  offsetcol = nothing, 
  report_keys = [:deviance, :dof_residual, :stderror, :vcov, :coef_table])

In [12]:
using GLM, MLJ
# Convertir 'y' a un vector unidimensional
y_vector = Vector{Float64}(y[!, 1])

# Entrenar el modelo
mach = machine(model_instance, X[train, :], y_vector[train])
fit!(mach)

# Evaluar el modelo
y_pred = MLJ.predict(mach, X[test, :])


[ Info: Training machine(LinearRegressor(fit_intercept = true, …), …).


52-element Vector{Normal{Float64}}:
 Normal{Float64}(μ=0.6188584351723111, σ=1.2860465066559517)
 Normal{Float64}(μ=1.3593881073941403, σ=1.2860465066559517)
 Normal{Float64}(μ=-1.0236123507793076, σ=1.2860465066559517)
 Normal{Float64}(μ=1.1559080486353976, σ=1.2860465066559517)
 Normal{Float64}(μ=2.3348624121301356, σ=1.2860465066559517)
 Normal{Float64}(μ=3.2709280883816643, σ=1.2860465066559517)
 Normal{Float64}(μ=4.71263421441663, σ=1.2860465066559517)
 Normal{Float64}(μ=-0.8147229433898175, σ=1.2860465066559517)
 Normal{Float64}(μ=1.9289937223737925, σ=1.2860465066559517)
 Normal{Float64}(μ=4.310272382629265, σ=1.2860465066559517)
 Normal{Float64}(μ=3.6312882572058665, σ=1.2860465066559517)
 Normal{Float64}(μ=5.535914501492529, σ=1.2860465066559517)
 Normal{Float64}(μ=1.6254365754881301, σ=1.2860465066559517)
 ⋮
 Normal{Float64}(μ=3.911885390762526, σ=1.2860465066559517)
 Normal{Float64}(μ=3.6487837579010365, σ=1.2860465066559517)
 Normal{Float64}(μ=5.423694938497659, σ=1.2860465

In [13]:
using Distributions
# Extraer las medias y las desviaciones estándar de las distribuciones normales
y_pred_means = mean.(y_pred)
y_pred_stds = std.(y_pred);

# Calcular el MSE
mse = mean((y_pred_means .- y_vector[test]).^2)
println("Error cuadrático medio para $target_cols_1: $mse")

Error cuadrático medio para ["Delta_Theta(º)"]: 1.6390619977792757


Dado que delta_theta es de este orden no parece predecir nada el modelo. Veamos otro.

In [17]:
using MLJ, MLJMultivariateStatsInterface
model_2 = @load RidgeRegressor pkg=MultivariateStats
model_instance_2 = RidgeRegressor(lambda = 0.5)

import MLJMultivariateStatsInterface ✔


[ Info: For silent loading, specify `verbosity=0`. 


RidgeRegressor(
  lambda = 0.5, 
  bias = true)

In [18]:
using GLM, MLJ
# Convertir 'y' a un vector unidimensional
y_vector = Vector{Float64}(y[!, 1])

# Entrenar el modelo
mach = machine(model_instance_2, X[train, :], y_vector[train])
fit!(mach)

# Evaluar el modelo
y_pred = MLJ.predict(mach, X[test, :])
mse = mean((y_pred .- y_vector[test]).^2)
println("Error cuadrático medio para $target_cols_1: $mse")

Error cuadrático medio para ["Delta_Theta(º)"]: 1.632420304777571


[ Info: Training machine(RidgeRegressor(lambda = 0.5, …), …).
